In [10]:
# para la lectura de datos
import pickle

# para el tratamiento de datos
import numpy as np
import pandas as pd

# para la lectura de todas las columnas/filas
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

# para los gráficos
import matplotlib.pyplot as plt
import seaborn as sns

# Encoding
from sklearn.preprocessing import OneHotEncoder

In [11]:
# Leer el archivo
df = pd.read_pickle("datos/2.bikes_corr.pkl")
df.head()

,dteday,seasons,yr,mnth,holiday_num,weekday_num,workingday_num,weathersit,atemp,casual,registered,cnt
instant,,,,,,,,,,,,
1,2018-01-01,winter,0,1,1,0,0,2,18.18125,331,654,985
2,2018-01-02,winter,0,1,0,1,1,2,17.68695,131,670,801
3,2018-01-03,winter,0,1,0,2,1,1,9.47025,120,1229,1349
4,2018-01-04,winter,0,1,0,3,1,1,10.60610,108,1454,1562
5,2018-01-05,winter,0,1,0,4,1,1,11.46350,82,1518,1600


In [12]:
# Leer el archivo entrenamiento Random Forest codificadas de casual
with open("datos/7.casual_encoding_oh_final_gs_rf.pkl", "rb") as f:
    rf_casual = pickle.load(f)

In [13]:
# iniciamos el método de OneHot Encoder
oh = OneHotEncoder()
transformados = oh.fit(df[['seasons']])
transformados_array = oh.transform(df[['seasons']])
transformados_df = pd.DataFrame(transformados_array.toarray())
transformados_df.columns = oh.get_feature_names_out()
df.drop(['seasons'], axis=1, inplace=True)
df = pd.concat([df, transformados_df], axis=1)
df.head()

,dteday,yr,mnth,holiday_num,weekday_num,workingday_num,weathersit,atemp,casual,registered,cnt,seasons_autumn,seasons_spring,seasons_summer,seasons_winter
1,2018-01-01,0.0,1.0,1.0,0.0,0.0,2.0,18.18125,331.0,654.0,985.0,0.0,0.0,0.0,1.0
2,2018-01-02,0.0,1.0,0.0,1.0,1.0,2.0,17.68695,131.0,670.0,801.0,0.0,0.0,0.0,1.0
3,2018-01-03,0.0,1.0,0.0,2.0,1.0,1.0,9.47025,120.0,1229.0,1349.0,0.0,0.0,0.0,1.0
4,2018-01-04,0.0,1.0,0.0,3.0,1.0,1.0,10.60610,108.0,1454.0,1562.0,0.0,0.0,0.0,1.0
5,2018-01-05,0.0,1.0,0.0,4.0,1.0,1.0,11.46350,82.0,1518.0,1600.0,0.0,0.0,0.0,1.0


In [14]:
# Predicción del usuario
# Vamos preguntando al usuario por pantalla la entrada de datos
holiday_usuario = int(input("¿Será festivo? 1-si, 0-no: "))
weekday_usuario = input("¿Qué día de la semana será? lunes, martes, miércoles, jueves, viernes, sabado, domingo: ").lower()
workingday_usuario = int(input("¿Será laborable? 1-si, 0-no: : "))
weathersit_usuario = int(input("""¿Cómo estará el día?\n
    (1-despejado/parcialmente nublado, 2-niebla, 3-nieve/lluvia ligera, 4-lluvia intensa/tormenta eléctrica): """))
temp_usuario = float(input("¿Qué temperatura hará?: "))
atemp_usuario = float(input("¿Qué sensación térmica hará?: "))
hum_usuario = float(input("¿Qué humedad hará?: "))
windspeed_usuario = float(input("¿Qué viento hará?: "))
season_usuario = input("¿Cuál será la estación? spring, summer, autumn, winter: ").lower()

# Almacenamos las entradas en un diccionario
usuario = {"holiday_num": holiday_usuario, "weekday_num": weekday_usuario, "workingday_num": workingday_usuario, 
    "weathersit": weathersit_usuario, "temp": temp_usuario, "atemp": atemp_usuario, 
    "hum": hum_usuario, "windspeed": windspeed_usuario, "seasons": season_usuario}

# Convertimos en un dataframe el diccionario
df_usuario = pd.DataFrame(usuario, index = [0])
df_usuario

,holiday_num,weekday_num,workingday_num,weathersit,temp,atemp,hum,windspeed,seasons
0,1,domingo,0,1,25.0,23.0,2.0,10.0,spring


In [15]:
# Primero transformar con One-Hot Encoder la columna seasons de usuario
df_onehot = pd.DataFrame(oh.transform(df_usuario[["seasons"]]).toarray(), columns=oh.get_feature_names_out())
df_usuario.drop(['seasons'], axis=1, inplace=True)
df_usuario = pd.concat([df_usuario, df_onehot], axis=1)
df_usuario

,holiday_num,weekday_num,workingday_num,weathersit,temp,atemp,hum,windspeed,seasons_autumn,seasons_spring,seasons_summer,seasons_winter
0,1,domingo,0,1,25.0,23.0,2.0,10.0,0.0,1.0,0.0,0.0


In [16]:
# Tercero convertir weekday, lunes = 0 hasta domingo = 6
# Definir el mapa a aplicar
mapa_weekday = {"lunes": 0, "martes": 1, "miercoles": 2, "jueves": 3, "viernes": 4,
    "sabado": 5, "domingo": 6}
# Aplicar el método map
df_usuario["weekday_num"] = df_usuario["weekday_num"].map(mapa_weekday) 
df_usuario

,holiday_num,weekday_num,workingday_num,weathersit,temp,atemp,hum,windspeed,seasons_autumn,seasons_spring,seasons_summer,seasons_winter
0,1,6,0,1,25.0,23.0,2.0,10.0,0.0,1.0,0.0,0.0


In [17]:
# convertimos las columnas string a category
df_usuario[["holiday_num", "weekday_num", "workingday_num", "weathersit", 'seasons_autumn', 'seasons_spring', 
    'seasons_summer', 'seasons_winter']] = df_usuario[["holiday_num", "weekday_num", "workingday_num", "weathersit", 
    'seasons_autumn', 'seasons_spring', 'seasons_summer', 'seasons_winter']].astype("category")
#df_usuario.dtypes

In [18]:
# Predicción de bicicletas para los usuarios casuales
rf_casual.predict(df_usuario)

array([1861.71074783])